In [1]:
import pandas as pd
import numpy as np
from numpy.random import RandomState
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from tabgan.sampler import  GANGenerator

c:\Users\DELL\AppData\Local\Programs\Python\Python39\lib\site-packages\_ctgan\synthesizer.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# df = pd.read_csv('C:\Pro\ANN\phong_gripper.csv', header= 0, index_col=0)  # type: ignore
# df.astype('float')
# rng = RandomState()

In [3]:
# model_data = df.sample(frac=0.8, random_state=rng)
# valid = df.loc[~df.index.isin(model_data.index)]

# x_train,x_test,y_train,y_test = train_test_split(model_data.iloc[:,:-4], 
# model_data.iloc[:,-4],test_size = 0.25)

# x_valid = valid.iloc[:,:-4]
# y_valid = valid.iloc[:,-4]

In [4]:
x_train = pd.read_csv('x_train.csv', header = 0, index_col=0)
y_train = pd.read_csv('y_train.csv', header=0, index_col=0)
x_test = pd.read_csv('x_test.csv', header=0, index_col=0)
y_test = pd.read_csv('y_test.csv', header=0, index_col=0)
x_valid = pd.read_csv('x_valid.csv', header=0, index_col=0)
y_valid = pd.read_csv('y_valid.csv', header=0, index_col=0)

In [5]:
# x_train = pd.DataFrame(x_train)
# y_train = pd.DataFrame(y_train)
test = pd.DataFrame(np.random.randint(0, 100, size=(int(20), 4)), columns=x_train.columns)

In [6]:
new_train2, new_target2 = GANGenerator(gen_x_times= int(1.37), cat_cols=None,bot_filter_quantile= 0, top_filter_quantile= 1,
                is_post_process=False,
            adversarial_model_params={
                "metrics": "rmse", "max_depth": 2, "max_bin": 100, 
                "learning_rate": 0.02, "random_state": 42, "n_estimators": 500,
            }, pregeneration_frac=1,only_generated_data= False,gen_params={"batch_size": 500, "epochs": 500, "patience": 25 }).generate_data_pipe(x_train, y_train, test, only_adversarial=False, deep_copy=True,use_adversarial=True )
    # print(new_train2, new_target2)


Fitting CTGAN transformers for each column:   0%|          | 0/5 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 16, number of negative: 16
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 32, number of used features: 0
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because 

In [7]:
scaler = StandardScaler()  
scaler.fit(new_train2)  
x_train = scaler.transform(new_train2)
x_test = scaler.transform(x_test)
y_test = y_test.to_numpy()

In [8]:
mlp = MLPRegressor(solver='adam', alpha=1e-5, max_iter= 10000,
                    hidden_layer_sizes=(10), random_state=1,
                    activation= 'logistic', early_stopping=True, n_iter_no_change=10)
mlp.fit(x_train,new_target2)

MLPRegressor(activation='logistic', alpha=1e-05, early_stopping=True,
             hidden_layer_sizes=10, max_iter=10000, random_state=1)

In [9]:
predict_test = mlp.predict(x_test)

In [10]:
predict_valid = mlp.predict(x_valid)

In [11]:
print(predict_test)

[1.27683795 1.23684283 1.25666946 1.28619285 1.23766926 1.30628642]


In [12]:
def rm_main():
    gan = pd.DataFrame(data = new_train2)
    gan['Expected'] = new_target2
    
    result = pd.DataFrame(data = x_valid)     
    result['Prediction'] = predict_valid
    result['Expected'] = y_valid    
    meta_data = dict(Prediction = ('numeric' , 'prediction'),
                     Expected = ('numeric' , 'label'))
    result.rm_metadata = meta_data   
    return result , gan

In [13]:
print(rm_main())

(    x1 (mm)    x2 (mm)   x3 (mm)   x4 (mm)   Prediction  Expected
#                                                                
11  7.932033   4.452582  0.940017  2.318829    1.231282  1.300422
15  7.932033   4.452582  1.082439  2.318829    1.228142  1.207579
19  7.932033   4.452582  0.940017  2.670157    1.228165  1.301454
22  6.888371   4.452582  1.082439  2.670157    1.227240  1.222227
24  6.888371   5.127196  1.082439  2.670157    1.230265  1.213776,     x1 (mm)    x2 (mm)   x3 (mm)   x4 (mm)   Expected
0   7.410202   5.268878  1.011228  2.494493  1.257523
1   7.410202   4.789889  1.112351  2.494493  1.200686
2   7.754130   4.498480  0.964845  2.626215  1.328548
3   7.726211   4.906063  1.145996  2.540141  1.217798
4   7.547876   5.252875  1.018768  2.562396  1.352364
5   6.799145   5.247118  1.047581  2.439706  1.316716
6   7.949467   4.954721  1.069046  2.571387  1.348433
7   7.695602   4.561631  1.004736  2.675975  1.286623
8   7.798216   5.122339  1.052006  2.384289  1.179